In [1]:
!ls

day_  prod  stage


In [2]:
!ls prod/nelson/common/

__pycache__  db_operations.py


In [3]:
#!/usr/bin/env python
# coding: utf-8
import sys
import platform
import logging

sys.path.append('./prod/nelson/')   #Comment this section when running on airflow
# sys.path.append('..') #comment this when running locally
from common import db_operations

import pandas as pd
import numpy as np
from datetime import datetime, date, timedelta
from common.db_operations import connect_to_trino, fetch_data_for_day, write_df_to_iceberg,execute_query

In [4]:
# Configure basic logging for the business logic file
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S')

# Print the Python version being used
print(f"Using Python version: {platform.python_version()}")

Using Python version: 3.11.13


In [7]:
conn = connect_to_trino()
query = f"""
select * 
FROM 
    adhoc.facts_prod.can_parsed_output_100
where id in ('3')
and dt = DATE '2025-09-10'
"""
df = execute_query(conn, query)
conn.close()
df.head()

2025-09-19 17:36:33 - INFO - 🔌 STEP 1: Connecting to Trino...
2025-09-19 17:36:33 - INFO - ✅ STEP 1: Connected to Trino
2025-09-19 17:36:33 - INFO - ⚙️ Executing query...
2025-09-19 17:37:00 - INFO - ✅ Query executed successfully!


,id,timestamp,insert_timestamp,sequence,number_of_can_ids,number_of_records,percentage_of_can_ids,lowpressureoilpumpfaultcode,lowvoltagedcaclifesignal,batterycoolingstate,...,dt,rank,ccs_spn3082_outcurrent_2,ccs_spn3081_outvolt_2,ccs_spn3082_outcurrent,ccs_spn3081_outvolt,bcl_spn3073_currtrequest2_2,bcl_spn3072_voltrequest2_2,bcl_spn3073_currtrequest2,bcl_spn3072_voltrequest2
0,3,2025-09-10 23:57:49.612,2025-09-10 23:58:10.061,51875,21,47,16.030533,0.0,45.0,None,...,2025-09-10,1,None,None,None,None,None,None,None,None
1,3,2025-09-10 23:58:44.411,2025-09-10 23:59:06.041,30033,20,61,15.267176,0.0,81.0,None,...,2025-09-10,1,None,None,None,None,None,None,None,None
2,3,2025-09-10 23:58:53.651,2025-09-10 23:59:16.204,37467,24,50,18.320612,0.0,174.0,None,...,2025-09-10,1,None,None,None,None,None,None,None,None
3,3,2025-09-10 23:59:01.791,2025-09-10 23:59:21.291,43931,20,50,15.267176,0.0,255.0,None,...,2025-09-10,1,None,None,None,None,None,None,None,None
4,3,2025-09-10 23:59:07.231,2025-09-10 23:59:31.459,48294,21,60,16.030533,0.0,53.0,None,...,2025-09-10,1,None,None,None,None,None,None,None,None


In [6]:
wait

NameError: name 'wait' is not defined

In [ ]:
# conn = connect_to_trino()
# # query = f"""
# # select * FROM adhoc.facts_prod.braking_report
# # where vehicle_id in ('3')
# # """
# query = """
# WITH two_days_data AS (
#     SELECT
#         "id", "timestamp", at_timezone("timestamp", 'Asia/Kolkata') AS IST, "BAT_SOC", "Bat_Voltage", "Total_Battery_Current", "GUN_Connection_Status", "OdoMeterReading", "Gear_Position", "Vehiclereadycondition"
#     FROM adhoc.facts_prod.can_parsed_output_100
#     WHERE
#         (dt = DATE '2025-09-10' OR dt = DATE '2025-09-10')
#         AND id IN ('3', '16', '18', '19')
# )
# SELECT
#     *
# FROM
#     two_days_data
# WHERE
#     CAST(IST AS DATE) = DATE '2025-09-10'   
# """
# df = execute_query(conn, query)
# conn.close()
# df.head()

In [ ]:
# ---- report configuration ----
TABLE_NAME = "energy_mileage_report"
SOURCE_TABLE = "can_parsed_output_100"
COLUMNS_TO_FETCH = [
    '"id"','"timestamp"',
    'at_timezone("timestamp", \'Asia/Kolkata\') AS IST',
    '"BAT_SOC"',
    '"Bat_Voltage"',
    '"Total_Battery_Current"',
    '"GUN_Connection_Status"',
    '"OdoMeterReading"',
    '"Gear_Position"',
    '"Vehiclereadycondition"']

In [ ]:
# --------------------
# Step 3: Function to process vehicle energy stats
# --------------------
def analyze_vehicle_energy_stats(df:pd.DataFrame):
    """
    Performs a combined analysis on an electric bus dataset that may contain
    data from multiple vehicles. The function calculates daily mileage, driving
    energy consumption, regenerative braking energy, and idling energy for
    each vehicle.
    """
    try:
        # Load the dataset
        # df = pd.read_csv(file_path)

        # --- General Data Preparation and Filtering ---
        # Ignore data points with extreme current values.
        df = df[df['Total_Battery_Current'].abs() <= 3000].copy()

        # Drop rows with missing data in key columns, including IST.
        # df.loc[:, 'IST'] = pd.to_datetime(df['timestamp'], unit='ms').dt.tz_localize('UTC').dt.tz_convert('Asia/Kolkata')
        df.dropna(subset=['OdoMeterReading', 'Gear_Position', 'Vehiclereadycondition', 'Total_Battery_Current', 'Bat_Voltage','IST'], inplace=True)

        # Sort data by vehicle ID and timestamp to ensure correct sequential calculations.
        df.sort_values(by=['id', 'IST'], inplace=True)

        # Extract the date for daily grouping
        df['date'] = df['IST'].dt.date

        # Get a list of unique vehicle IDs to iterate through
        unique_ids = df['id'].unique()
        all_daily_stats = []

        print("\n--- Daily Vehicle Statistics (Driving, Regenerative Braking, and Idling) ---")

        # Iterate over each unique vehicle ID
        for vehicle_id in unique_ids:
            vehicle_df = df[df['id'] == vehicle_id].copy()

            # Check if the grouped DataFrame is empty
            if vehicle_df.empty:
                print(f"No charging events were detected for device {vehicle_id}.")
                continue  

            # Calculate the time difference between consecutive data points for the current vehicle.
            vehicle_df['time_diff_seconds'] = vehicle_df['IST'].diff().dt.total_seconds().fillna(0)

            # Filter out records where time difference is zero or negative.
            vehicle_df = vehicle_df[vehicle_df['time_diff_seconds'] > 0]

            # Calculate the power in kW
            vehicle_df['power_kW'] = (vehicle_df['Bat_Voltage'] * vehicle_df['Total_Battery_Current']) / 1000

            # --- Analysis 1: Daily Driving and Regenerative Braking Statistics ---
            driving_df = vehicle_df[vehicle_df['Gear_Position'] == 2.0].copy()

            # Calculate energy consumption (positive power) and regenerative braking energy (negative power)
            driving_df['energy_consumption_kwh'] = driving_df.apply(
                lambda row: row['power_kW'] * (row['time_diff_seconds'] / 3600) if row['power_kW'] > 0 else 0, axis=1)
            driving_df['regen_energy_kwh'] = driving_df.apply(
                lambda row: -row['power_kW'] * (row['time_diff_seconds'] / 3600) if row['power_kW'] < 0 else 0, axis=1)

            # Calculate incremental mileage for each row.
            driving_df['distance_increment'] = driving_df['OdoMeterReading'].diff().fillna(0)

            # Filter out unrealistic mileage jumps (e.g., > 10 km in a single time step)
            driving_df = driving_df[driving_df['distance_increment'] <= 10]

            if not driving_df.empty:
                grouped_by_day_driving = driving_df.groupby(['id', 'date'])
                daily_driving_mileage = grouped_by_day_driving['distance_increment'].sum()
                daily_driving_energy = grouped_by_day_driving['energy_consumption_kwh'].sum()
                daily_regen_energy = grouped_by_day_driving['regen_energy_kwh'].sum()

                daily_stats_df = pd.DataFrame({
                    'dist_travelled_km': round(daily_driving_mileage, 2), #Mileage in km
                    'energy_consumed_kwh': round(daily_driving_energy, 2), #Daily Driving Energy Consumed (kWh)
                    'regen_energy_kwh': round(daily_regen_energy, 2) #Daily Regenerative Braking Energy (kWh)
                }).reset_index()
            else:
                # Create an empty DataFrame with the correct columns if there is no driving data
                daily_stats_df = pd.DataFrame(columns=[
                    'id', 'date', 'dist_travelled_km',
                    'energy_consumed_kwh',
                    'regen_energy_kwh'
                ])

            # --- Analysis 2: Daily Idling Energy Consumption ---
            # Filter for idling periods using 'Gear_Position' == 0 (Neutral) and 'Vehiclereadycondition' == 1
            stationary_df = vehicle_df[(vehicle_df['Gear_Position'] == 0.0) & 
                                       (vehicle_df['Vehiclereadycondition'] == 1.0)].copy()

            if not stationary_df.empty:
                stationary_df['energy_kwh'] = stationary_df.apply(
                    lambda row: row['power_kW'] * (row['time_diff_seconds'] / 3600) if row['power_kW'] > 0 else 0, axis=1)
                grouped_by_day_stationary = stationary_df.groupby(['id', 'date'])
                daily_idling_energy = grouped_by_day_stationary['energy_kwh'].sum()

                daily_idling_df = daily_idling_energy.reset_index(name='idling_energy_kwh') #Daily Idling Energy Consumed (kWh)
                daily_idling_df = daily_idling_df.round({'idling_energy_kwh': 2})  

                # Merge with existing stats
                daily_stats_df = pd.merge(daily_stats_df, daily_idling_df, on=['id', 'date'], how='outer')
                daily_stats_df = daily_stats_df.fillna(0)
            else:
                # If there's only driving data and no idling data
                if not daily_stats_df.empty:
                    daily_stats_df['idling_energy_kwh'] = 0

            if not daily_stats_df.empty:
                #Net Energy Consumed (kWh)
                daily_stats_df['net_energy_kwh'] = daily_stats_df['energy_consumed_kwh'] - daily_stats_df['regen_energy_kwh']
                #Net Consumption Rate (kWh/km)
                daily_stats_df['mileage_kwh_per_km'] = (daily_stats_df['net_energy_kwh'] / daily_stats_df['dist_travelled_km']).round(2)
                all_daily_stats.append(daily_stats_df)
        
        if all_daily_stats:            
            final_df = pd.concat(all_daily_stats, ignore_index=True)
            final_df = final_df[final_df.mileage_kwh_per_km.isna()==False]
            final_df.rename(columns={'id': 'vehicle_id'}, inplace=True)
            return final_df
        else:
            print("No data found for driving or idling periods.")
            return pd.DataFrame()

    except Exception as e:
        print(f"An error occurred: {e}")
        return pd.DataFrame()

In [ ]:
def process_data(conn,df:pd.DataFrame):
    # Process data related energy-mileage relation
    df_res = df
    if not df_res.empty:
        # write_df_to_iceberg(conn, df_res, "energy_mileage_report" )
        logging.info("✅ Processing and write for specific IDs complete.")
    else:
        logging.info("Processed DataFrame is empty. No data to write.")
    return df_res

In [ ]:
conn = connect_to_trino()
start_date_str = '2025-09-10'
start_date = date.fromisoformat(start_date_str)
print(start_date.isoformat())
vehicle_ids_for_report = ['3', '16', '18', '19']
df_duplicate_raw = fetch_data_for_day(conn, start_date.isoformat(), COLUMNS_TO_FETCH, SOURCE_TABLE, vehicle_ids_for_report)
df_duplicate_raw.head()

2025-09-19 08:34:05 - INFO - 🔌 STEP 1: Connecting to Trino...
2025-09-19 08:34:05 - INFO - ✅ STEP 1: Connected to Trino
2025-09-19 08:34:05 - INFO - 📥 STEP 2a: Validating and fetching data for 2025-09-10...
2025-09-19 08:34:05 - INFO - 
    WITH two_days_data AS (
        SELECT
            "id", "timestamp", at_timezone("timestamp", 'Asia/Kolkata') AS IST, "BAT_SOC", "Bat_Voltage", "Total_Battery_Current", "GUN_Connection_Status", "OdoMeterReading", "Gear_Position", "Vehiclereadycondition"
        FROM adhoc.facts_prod.can_parsed_output_100
        WHERE
            (dt = DATE '2025-09-10' OR dt = DATE '2025-09-09')
            AND id IN ('3', '16', '18', '19')
    )
    SELECT
        *
    FROM
        two_days_data
    WHERE
        CAST(IST AS DATE) = DATE '2025-09-10'    
    
2025-09-19 08:34:05 - INFO - ⚙️ Executing query...


2025-09-10


2025-09-19 08:34:21 - INFO - ✅ Query executed successfully!
2025-09-19 08:34:21 - INFO - ✅ STEP 2d: Data fetching for 2025-09-10 completed, Rows fetched: 91192


,id,timestamp,IST,BAT_SOC,Bat_Voltage,Total_Battery_Current,GUN_Connection_Status,OdoMeterReading,Gear_Position,Vehiclereadycondition
0,16,2025-09-10 18:23:04.832,2025-09-10 23:53:04.832000+05:30,78.4,633.3,87.2,0.0,14396.000,2.0,1.0
1,16,2025-09-10 18:23:10.132,2025-09-10 23:53:10.132000+05:30,78.4,633.0,101.4,0.0,14396.125,2.0,1.0
2,16,2025-09-10 18:23:24.573,2025-09-10 23:53:24.573000+05:30,78.4,632.9,105.6,0.0,14396.500,2.0,1.0
3,16,2025-09-10 18:23:51.893,2025-09-10 23:53:51.893000+05:30,78.4,633.1,85.3,0.0,NaN,2.0,1.0
4,16,2025-09-10 18:23:55.152,2025-09-10 23:53:55.152000+05:30,78.4,633.0,97.8,0.0,NaN,2.0,1.0


In [ ]:
# --------------------
# Main execution logic
# --------------------
def main(start_date_str: str = None, end_date_str: str = None):
    conn = connect_to_trino()
    df_duplicate_processed = pd.DataFrame()
    df_duplicate_raw = pd.DataFrame()
    vehicle_ids_for_report = []    
    if conn:
        try:
            # Determine the date range to process
            if start_date_str and end_date_str:
                start_date = date.fromisoformat(start_date_str)
                end_date = date.fromisoformat(end_date_str)
                date_range = [start_date + timedelta(days=i) for i in range((end_date - start_date).days + 1)]
            else:
                # Default to processing yesterday's data
                date_range = [date.today() - timedelta(days=1)]


            for single_date in date_range:
                date_str = single_date.isoformat()
                logging.info(f"▶️ Starting daily report job for {date_str}")

                # Example 2: Call the function with specific vehicle IDs
                logging.info("\n--- Processing specific vehicle IDs ---")
                vehicle_ids_for_report = ['3', '16', '18', '19']
                df_duplicate_raw = fetch_data_for_day(conn, date_str, COLUMNS_TO_FETCH, SOURCE_TABLE, vehicle_ids_for_report)
                

                # if not df_duplicate_raw.empty:
                #     df_duplicate_processed = process_data(conn,analyze_vehicle_energy_stats(df_raw_specific))
                # else:
                #     logging.info("Raw DataFrame is empty. No processing needed.")

        except Exception as e:
            logging.critical(f"❌ A critical error occurred in the main script: {e}")

        finally:
            logging.info("🔒 STEP 5: Closing Trino connection...")
            conn.close()
            logging.info("✅ STEP 5: Connection closed.")
    else:
        logging.critical("❌ Failed to establish a database connection. Exiting.")
    
    return df_duplicate_raw, df_duplicate_processed

In [ ]:

if __name__ == "__main__":
    global_df_raw, global_df_processed = main()
    # --- For a one-time manual backfill, uncomment the line below and set your dates ---
    # main(start_date_str='2025-07-12', end_date_str='2025-09-15')

    # --- For daily automated runs, use the existing call ---
    # main()

2025-09-19 08:34:22 - INFO - 🔌 STEP 1: Connecting to Trino...
2025-09-19 08:34:22 - INFO - ✅ STEP 1: Connected to Trino
2025-09-19 08:34:22 - INFO - ▶️ Starting daily report job for 2025-09-18
2025-09-19 08:34:22 - INFO - 
--- Processing specific vehicle IDs ---
2025-09-19 08:34:22 - INFO - 📥 STEP 2a: Validating and fetching data for 2025-09-18...
2025-09-19 08:34:22 - INFO - 
    WITH two_days_data AS (
        SELECT
            "id", "timestamp", at_timezone("timestamp", 'Asia/Kolkata') AS IST, "BAT_SOC", "Bat_Voltage", "Total_Battery_Current", "GUN_Connection_Status", "OdoMeterReading", "Gear_Position", "Vehiclereadycondition"
        FROM adhoc.facts_prod.can_parsed_output_100
        WHERE
            (dt = DATE '2025-09-18' OR dt = DATE '2025-09-17')
            AND id IN ('3', '16', '18', '19')
    )
    SELECT
        *
    FROM
        two_days_data
    WHERE
        CAST(IST AS DATE) = DATE '2025-09-18'    
    
2025-09-19 08:34:22 - INFO - ⚙️ Executing query...
2025-09-19 0

In [ ]:
global_df_raw.head()

,id,timestamp,IST,BAT_SOC,Bat_Voltage,Total_Battery_Current,GUN_Connection_Status,OdoMeterReading,Gear_Position,Vehiclereadycondition
0,16,2025-09-18 17:49:44.825,2025-09-18 23:19:44.825000+05:30,86.8,630.0,298.1,0.0,19636.125,2.0,1.0
1,16,2025-09-18 17:49:48.025,2025-09-18 23:19:48.025000+05:30,86.8,629.3,288.7,0.0,NaN,2.0,1.0
2,16,2025-09-18 17:37:24.983,2025-09-18 23:07:24.983000+05:30,NaN,NaN,NaN,0.0,19627.750,2.0,1.0
3,16,2025-09-18 17:37:35.163,2025-09-18 23:07:35.163000+05:30,88.4,633.5,106.9,0.0,19627.875,2.0,1.0
4,16,2025-09-18 17:37:36.963,2025-09-18 23:07:36.963000+05:30,88.4,633.9,95.7,0.0,19627.875,2.0,1.0


In [ ]:
print(global_df_raw.isnull().sum())


id                           0
timestamp                    0
IST                          0
BAT_SOC                  26884
Bat_Voltage              26884
Total_Battery_Current    26884
GUN_Connection_Status     9952
OdoMeterReading          58182
Gear_Position            10075
Vehiclereadycondition    10075
date                         0
dtype: int64


In [ ]:
# Show rows with any null values
null_rows = global_df_raw[global_df_raw.isnull().any(axis=1)]
print(null_rows[null_rows.OdoMeterReading.isnull()==False].size)
print(global_df_raw[global_df_raw.isnull().any(axis=1)].size)
# null_rows[null_rows.GUN_Connection_Status==1]
# null_rows.size


168058
808060


In [ ]:
global_df_raw.loc[:, 'date'] = global_df_raw['IST'].dt.date

In [ ]:
global_df_raw.to_csv('day_')

In [ ]:
# Sort by 'count_of_instances' in descending order
grouped_df_sorted = global_df_raw.groupby(['id', 'date']).size().reset_index(name='count_of_instances').sort_values(by='id', ascending=False)
grouped_df_sorted
# grouped_df_sorted = grouped_df.sort_values(by='count_of_instances', ascending=False)
# grouped_df_sorted = grouped_df.sort_values(by=['date', 'count_of_instances'], ascending=[True, False])

,id,date,count_of_instances
3,3,2025-09-18,47760
2,19,2025-09-18,31852
1,18,2025-09-18,3468
0,16,2025-09-18,45927


In [ ]:
global_df_raw.to_csv('data_180925.csv')

In [ ]:
global_df_processed.head()

""
